In [5]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

In [13]:
DATA_SAMPLE_PATH = 'data_sample'
ECG_FILE_PATH = os.path.join(DATA_SAMPLE_PATH, 'ECG_II-20240912092357~20240912102356.csv')

In [23]:
ecg = pd.read_csv(ECG_FILE_PATH,
                  header=None,
                  names=['datetime'] + list(range(500)),
                  parse_dates=["datetime"])

In [24]:
ecg['datetime'].diff().describe()

count               3599
mean     0 days 00:00:01
std      0 days 00:00:00
min      0 days 00:00:01
25%      0 days 00:00:01
50%      0 days 00:00:01
75%      0 days 00:00:01
max      0 days 00:00:01
Name: datetime, dtype: object

In [27]:
ecg['datetime'].diff().iloc[-1].total_seconds()

1.0